In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import dill as pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import FalseNegatives

In [2]:
def load_sentence_embeddings(model='DistilBERT', features_path='features/', filename='sentence_embeddings'):
    if model == 'DistilBERT':
        tst_df = pd.concat([pickle.load(open(os.path.join(features_path, file), 'rb')) for file in os.listdir(features_path) if filename in file])
        return tst_df

In [3]:
tst_df = load_sentence_embeddings()

In [4]:
tst_df.shape

(22332, 2)

In [5]:
tst_df.head()

,sentence_embeddings,label
0,"[-0.21086546778678894, -0.005486507900059223, ...",0
1,"[-0.3340323865413666, 0.1280461549758911, -0.6...",0
2,"[-0.4694300889968872, -0.14065003395080566, -0...",0
3,"[-0.19011789560317993, 0.16007745265960693, -0...",0
4,"[-0.5151359438896179, -0.09672432392835617, -0...",0


In [6]:
features = np.array(tst_df.sentence_embeddings.tolist())
labels = tst_df['label'].values

In [7]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [8]:
model = Sequential()
model.add(Dense(12, input_shape=(768, ), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_m])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                9228      
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 9341 (36.49 KB)
Trainable params: 9341 (36.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
#model.fit(features, labels, epochs=150, batch_size=100)

In [11]:
model.fit(features, labels, epochs=150, batch_size=100, workers=32, use_multiprocessing=True)

Epoch 1/150
224/224 [==============================] - 2s 3ms/step - loss: 0.2967 - f1_m: 0.7530
Epoch 2/150
224/224 [==============================] - 1s 3ms/step - loss: 0.1279 - f1_m: 0.9303
Epoch 3/150
224/224 [==============================] - 1s 3ms/step - loss: 0.1006 - f1_m: 0.9448
Epoch 4/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0832 - f1_m: 0.9559
Epoch 5/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0757 - f1_m: 0.9584
Epoch 6/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0684 - f1_m: 0.9639
Epoch 7/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0656 - f1_m: 0.9653
Epoch 8/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0588 - f1_m: 0.9696
Epoch 9/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0562 - f1_m: 0.9701
Epoch 10/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0538 - f1_m: 0.9721
Epoch 11/150
224/224 [=======

224/224 [==============================] - 1s 3ms/step - loss: 0.0157 - f1_m: 0.9917
Epoch 85/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0166 - f1_m: 0.9904
Epoch 86/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0192 - f1_m: 0.9883
Epoch 87/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0144 - f1_m: 0.9923
Epoch 88/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0166 - f1_m: 0.9911
Epoch 89/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0155 - f1_m: 0.9920
Epoch 90/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0165 - f1_m: 0.9903
Epoch 91/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0139 - f1_m: 0.9921
Epoch 92/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0138 - f1_m: 0.9933
Epoch 93/150
224/224 [==============================] - 1s 3ms/step - loss: 0.0153 - f1_m: 0.9919
Epoch 94/150
224/224 [===========

In [12]:
loss, f1_score = model.evaluate(features, labels, verbose=0)

In [15]:
loss, f1_score

(0.0040576765313744545, 0.33626389503479004)

In [16]:
predictions = (model.predict(features) > 0.5).astype(int)


698/698 [==============================] - 1s 2ms/step


In [17]:
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        print(i, predictions[i])

2359 [0]
5465 [0]
5491 [0]
5505 [0]
5559 [0]
5613 [0]
5653 [0]
5661 [0]
5813 [0]
5946 [0]
5993 [0]
6162 [0]
6184 [0]
12416 [0]
14890 [0]
14892 [0]
14893 [0]
15174 [0]
17990 [1]
19796 [1]
20878 [1]
21221 [1]
21709 [1]
21790 [1]
22023 [1]
22315 [1]


In [18]:
score = model.evaluate(test_features, test_labels, verbose=0)

NameError: name 'test_features' is not defined

In [20]:
model = Sequential()
model.add(Dense(12, input_shape=(768, ), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', FalseNegatives()])

In [13]:
accuracy

[0.9998656511306763, 3.0]

In [ ]:

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
